# Filtering Unneeded Openings

## Purpose
- To understand which openings in the DB may not be helpful for processing.

## Possible Filters
- Very large or small win rates - likely just a bad opening for one side, no need to recommend it
- Extremely rare openings - not enough data to give recommendations


In [ ]:
# Configuration and setup
import pandas as pd
from pathlib import Path
from utils.database.db_utils import get_db_connection

# Define the path to the DuckDB database file
project_root = Path.cwd().parent if "notebooks" in str(Path.cwd()) else Path.cwd()
db_path = project_root / "data" / "processed" / "chess_games.db"

# Set pandas display options for better readability
pd.set_option('display.float_format', '{:,.2f}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print(f"Database path: {db_path}")
print(f"Database exists: {db_path.exists()}")

# Configuration: adjust these numbers to control how many results to show
TOP_N_LEAST_PLAYED_BY_GAMES = 100      # Show N least played openings by total games
TOP_N_LEAST_PLAYED_BY_PLAYERS = 200    # Show N least played openings by unique players  
TOP_N_HIGHEST_SCORING = 15             # Show N highest scoring openings (best win rates)
TOP_N_LOWEST_SCORING = 15              # Show N lowest scoring openings (worst win rates)

print(f"\nAnalysis Configuration:")
print(f"- Least played by games: {TOP_N_LEAST_PLAYED_BY_GAMES}")
print(f"- Least played by players: {TOP_N_LEAST_PLAYED_BY_PLAYERS}")
print(f"- Highest scoring openings: {TOP_N_HIGHEST_SCORING}")
print(f"- Lowest scoring openings: {TOP_N_LOWEST_SCORING}")

Database path: /Users/a/Documents/personalprojects/chess-opening-recommender/data/processed/chess_games.db
Database exists: True

Analysis Configuration:
- Least played by games: 100
- Least played by players: 20
- Highest scoring openings: 15
- Lowest scoring openings: 15


## 1. Database Overview

First, let's get a high-level overview of our database to understand the scale of data we're working with.

In [10]:
# Database overview - basic counts
if db_path.exists():
    with get_db_connection(db_path) as con:
        print("=== DATABASE OVERVIEW ===")
        
        # Count total records in each table
        player_count = con.execute('SELECT COUNT(*) FROM player').fetchone()[0]
        opening_count = con.execute('SELECT COUNT(*) FROM opening').fetchone()[0]
        
        print(f"Total Players: {player_count:,}")
        print(f"Total Openings: {opening_count:,}")
        
        # Count total games across all partitions
        total_games = con.execute("""
            SELECT SUM(num_wins + num_draws + num_losses) as total_games
            FROM player_opening_stats
        """).fetchone()[0]
        
        total_stats_records = con.execute('SELECT COUNT(*) FROM player_opening_stats').fetchone()[0]
        
        print(f"Total Games: {total_games:,}")
        print(f"Total Player-Opening-Color Records: {total_stats_records:,}")
        print(f"Average Games per Record: {total_games/total_stats_records:.1f}")
        
        # Show partition distribution
        print("\n--- Partition Distribution ---")
        partitions = ['A', 'B', 'C', 'D', 'E', 'other']
        partition_data = []
        
        for partition in partitions:
            count = con.execute(f'SELECT COUNT(*) FROM player_opening_stats_{partition}').fetchone()[0]
            partition_data.append({'Partition': partition, 'Records': count})
            
        partition_df = pd.DataFrame(partition_data)
        partition_df['Percentage'] = (partition_df['Records'] / partition_df['Records'].sum() * 100).round(2)
        partition_df['Records'] = partition_df['Records'].apply('{:,}'.format)
        partition_df['Percentage'] = partition_df['Percentage'].apply('{:.2f}%'.format)
        
        print(partition_df.to_string(index=False))
else:
    print(f"Database file not found at {db_path}")

=== DATABASE OVERVIEW ===
Total Players: 50,000
Total Openings: 3,593
Total Games: 568,894,735
Total Player-Opening-Color Records: 28,229,204
Average Games per Record: 20.2

--- Partition Distribution ---
Partition   Records Percentage
        A 6,721,561     23.81%
        B 7,292,844     25.83%
        C 9,221,077     32.67%
        D 3,952,764     14.00%
        E 1,040,958      3.69%
    other         0      0.00%


## 2. Least Played Openings by Total Games

These are openings that have very few total games played across all players. They might be too rare to provide meaningful recommendations.

In [11]:
# Least played openings by total number of games
if db_path.exists():
    with get_db_connection(db_path) as con:
        print(f"=== TOP {TOP_N_LEAST_PLAYED_BY_GAMES} LEAST PLAYED OPENINGS BY TOTAL GAMES ===")
        
        least_played_by_games = con.execute(f"""
            SELECT 
                o.eco,
                o.name,
                SUM(pos.num_wins + pos.num_draws + pos.num_losses) as total_games,
                COUNT(DISTINCT pos.player_id) as unique_players,
                COUNT(DISTINCT CASE WHEN pos.color = 'w' THEN pos.player_id END) as white_players,
                COUNT(DISTINCT CASE WHEN pos.color = 'b' THEN pos.player_id END) as black_players,
                SUM(pos.num_wins) as total_wins,
                SUM(pos.num_draws) as total_draws, 
                SUM(pos.num_losses) as total_losses,
                ROUND(SUM(pos.num_wins) * 100.0 / NULLIF(SUM(pos.num_wins + pos.num_draws + pos.num_losses), 0), 2) as win_percentage,
                ROUND(SUM(pos.num_draws) * 100.0 / NULLIF(SUM(pos.num_wins + pos.num_draws + pos.num_losses), 0), 2) as draw_percentage,
                ROUND(SUM(pos.num_losses) * 100.0 / NULLIF(SUM(pos.num_wins + pos.num_draws + pos.num_losses), 0), 2) as loss_percentage
            FROM opening o
            JOIN player_opening_stats pos ON o.id = pos.opening_id
            GROUP BY o.id, o.eco, o.name
            ORDER BY total_games ASC
            LIMIT {TOP_N_LEAST_PLAYED_BY_GAMES}
        """).fetchdf()
        
        # Format the display
        display_df = least_played_by_games.copy()
        display_df['total_games'] = display_df['total_games'].apply('{:,}'.format)
        display_df['unique_players'] = display_df['unique_players'].apply('{:,}'.format)
        display_df['white_players'] = display_df['white_players'].apply('{:,}'.format)
        display_df['black_players'] = display_df['black_players'].apply('{:,}'.format)
        display_df['total_wins'] = display_df['total_wins'].apply('{:,}'.format)
        display_df['total_draws'] = display_df['total_draws'].apply('{:,}'.format)
        display_df['total_losses'] = display_df['total_losses'].apply('{:,}'.format)
        
        # Rename columns for better display
        display_df.columns = ['ECO', 'Opening Name', 'Total Games', 'Unique Players', 
                             'White Players', 'Black Players', 'Wins', 'Draws', 'Losses',
                             'Win %', 'Draw %', 'Loss %']
        
        print(display_df.to_string(index=False))
        
        # Summary stats
        total_games_in_results = least_played_by_games['total_games'].sum()
        avg_games_per_opening = least_played_by_games['total_games'].mean()
        median_games_per_opening = least_played_by_games['total_games'].median()
        
        print(f"\n--- Summary Statistics ---")
        print(f"Total games in these {TOP_N_LEAST_PLAYED_BY_GAMES} openings: {total_games_in_results:,}")
        print(f"Average games per opening: {avg_games_per_opening:.1f}")
        print(f"Median games per opening: {median_games_per_opening:.1f}")
        print(f"Min games: {least_played_by_games['total_games'].min():,}")
        print(f"Max games: {least_played_by_games['total_games'].max():,}")

=== TOP 100 LEAST PLAYED OPENINGS BY TOTAL GAMES ===


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

ECO                                                                  Opening Name Total Games Unique Players White Players Black Players Wins Draws Losses  Win %  Draw %  Loss %
C12                    French Defense: McCutcheon Variation, Bogoljubow Variation         1.0              1             0             1  1.0   0.0    0.0 100.00    0.00    0.00
A00                                           Van Geet Opening: Jendrossek Gambit         1.0              1             1             0  0.0   0.0    1.0   0.00    0.00  100.00
C39  King's Gambit Accepted: Kieseritzky Gambit, Brentano Defense, Caro Variation         1.0              1             0             1  1.0   0.0    0.0 100.00    0.00    0.00
C88                                            Ruy Lopez: Closed, Alekhine Gambit         1.0              1             1             0  1.0   0.0    0.0 100.00    0.00    0.00
C51                         Italian Game: Evans Gambit Declined, Hicken Variation         1.0              1  

## 3. Least Played Openings by Number of Players

These openings have been played by very few unique players, which might indicate they're too specialized or obscure for general recommendations.

In [12]:
# Least played openings by number of unique players
if db_path.exists():
    with get_db_connection(db_path) as con:
        print(f"=== TOP {TOP_N_LEAST_PLAYED_BY_PLAYERS} LEAST PLAYED OPENINGS BY UNIQUE PLAYERS ===")
        
        least_played_by_players = con.execute(f"""
            SELECT 
                o.eco,
                o.name,
                COUNT(DISTINCT pos.player_id) as unique_players,
                SUM(pos.num_wins + pos.num_draws + pos.num_losses) as total_games,
                COUNT(DISTINCT CASE WHEN pos.color = 'w' THEN pos.player_id END) as white_players,
                COUNT(DISTINCT CASE WHEN pos.color = 'b' THEN pos.player_id END) as black_players,
                ROUND(AVG(pos.num_wins + pos.num_draws + pos.num_losses), 1) as avg_games_per_player,
                SUM(pos.num_wins) as total_wins,
                SUM(pos.num_draws) as total_draws,
                SUM(pos.num_losses) as total_losses,
                ROUND(SUM(pos.num_wins) * 100.0 / NULLIF(SUM(pos.num_wins + pos.num_draws + pos.num_losses), 0), 2) as win_percentage,
                ROUND(SUM(pos.num_draws) * 100.0 / NULLIF(SUM(pos.num_wins + pos.num_draws + pos.num_losses), 0), 2) as draw_percentage,
                ROUND(SUM(pos.num_losses) * 100.0 / NULLIF(SUM(pos.num_wins + pos.num_draws + pos.num_losses), 0), 2) as loss_percentage
            FROM opening o
            JOIN player_opening_stats pos ON o.id = pos.opening_id
            GROUP BY o.id, o.eco, o.name
            ORDER BY unique_players ASC, total_games ASC
            LIMIT {TOP_N_LEAST_PLAYED_BY_PLAYERS}
        """).fetchdf()
        
        # Format the display
        display_df = least_played_by_players.copy()
        display_df['unique_players'] = display_df['unique_players'].apply('{:,}'.format)
        display_df['total_games'] = display_df['total_games'].apply('{:,}'.format)
        display_df['white_players'] = display_df['white_players'].apply('{:,}'.format)
        display_df['black_players'] = display_df['black_players'].apply('{:,}'.format)
        display_df['total_wins'] = display_df['total_wins'].apply('{:,}'.format)
        display_df['total_draws'] = display_df['total_draws'].apply('{:,}'.format)
        display_df['total_losses'] = display_df['total_losses'].apply('{:,}'.format)
        
        # Rename columns for better display  
        display_df.columns = ['ECO', 'Opening Name', 'Unique Players', 'Total Games',
                             'White Players', 'Black Players', 'Avg Games/Player', 'Wins', 'Draws', 'Losses',
                             'Win %', 'Draw %', 'Loss %']
        
        print(display_df.to_string(index=False))
        
        # Summary stats
        total_players_in_results = least_played_by_players['unique_players'].sum()
        total_games_in_results = least_played_by_players['total_games'].sum()
        avg_players_per_opening = least_played_by_players['unique_players'].mean()
        median_players_per_opening = least_played_by_players['unique_players'].median()
        
        print(f"\n--- Summary Statistics ---")
        print(f"Total unique players across these {TOP_N_LEAST_PLAYED_BY_PLAYERS} openings: {total_players_in_results:,}")
        print(f"Total games in these openings: {total_games_in_results:,}")
        print(f"Average players per opening: {avg_players_per_opening:.1f}")
        print(f"Median players per opening: {median_players_per_opening:.1f}")
        print(f"Min players: {least_played_by_players['unique_players'].min():,}")
        print(f"Max players: {least_played_by_players['unique_players'].max():,}")

=== TOP 20 LEAST PLAYED OPENINGS BY UNIQUE PLAYERS ===


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

ECO                                                                  Opening Name Unique Players Total Games White Players Black Players  Avg Games/Player Wins Draws Losses  Win %  Draw %  Loss %
A00                                               Hungarian Opening: Asten Gambit              1         1.0             1             0              1.00  0.0   0.0    1.0   0.00    0.00  100.00
C80                                               Ruy Lopez: Open, Harksen Gambit              1         1.0             0             1              1.00  1.0   0.0    0.0 100.00    0.00    0.00
D99                       Grünfeld Defense: Russian Variation, Yugoslav Variation              1         1.0             1             0              1.00  0.0   0.0    1.0   0.00    0.00  100.00
C51                         Italian Game: Evans Gambit Declined, Hicken Variation              1         1.0             0             1              1.00  0.0   0.0    1.0   0.00    0.00  100.00
B06                 

## 4. Highest Scoring Openings (Best Win Rates)

These openings have the highest win rates, which might indicate they're unbalanced or too situational to be useful for general recommendations.

In [13]:
# Highest scoring openings (best win rates) - only include openings with meaningful sample sizes
if db_path.exists():
    with get_db_connection(db_path) as con:
        print(f"=== TOP {TOP_N_HIGHEST_SCORING} HIGHEST SCORING OPENINGS (MIN 100 GAMES) ===")
        
        highest_scoring = con.execute(f"""
            SELECT 
                o.eco,
                o.name,
                SUM(pos.num_wins + pos.num_draws + pos.num_losses) as total_games,
                COUNT(DISTINCT pos.player_id) as unique_players,
                COUNT(DISTINCT CASE WHEN pos.color = 'w' THEN pos.player_id END) as white_players,
                COUNT(DISTINCT CASE WHEN pos.color = 'b' THEN pos.player_id END) as black_players,
                SUM(pos.num_wins) as total_wins,
                SUM(pos.num_draws) as total_draws,
                SUM(pos.num_losses) as total_losses,
                ROUND(SUM(pos.num_wins) * 100.0 / NULLIF(SUM(pos.num_wins + pos.num_draws + pos.num_losses), 0), 2) as win_percentage,
                ROUND(SUM(pos.num_draws) * 100.0 / NULLIF(SUM(pos.num_wins + pos.num_draws + pos.num_losses), 0), 2) as draw_percentage,
                ROUND(SUM(pos.num_losses) * 100.0 / NULLIF(SUM(pos.num_wins + pos.num_draws + pos.num_losses), 0), 2) as loss_percentage,
                -- Score calculation: (wins + 0.5*draws) / total_games * 100
                ROUND((SUM(pos.num_wins) + 0.5 * SUM(pos.num_draws)) * 100.0 / NULLIF(SUM(pos.num_wins + pos.num_draws + pos.num_losses), 0), 2) as score_percentage
            FROM opening o
            JOIN player_opening_stats pos ON o.id = pos.opening_id
            GROUP BY o.id, o.eco, o.name
            HAVING SUM(pos.num_wins + pos.num_draws + pos.num_losses) >= 100  -- Only consider openings with meaningful sample size
            ORDER BY score_percentage DESC, total_games DESC
            LIMIT {TOP_N_HIGHEST_SCORING}
        """).fetchdf()
        
        # Format the display
        display_df = highest_scoring.copy()
        display_df['total_games'] = display_df['total_games'].apply('{:,}'.format)
        display_df['unique_players'] = display_df['unique_players'].apply('{:,}'.format)
        display_df['white_players'] = display_df['white_players'].apply('{:,}'.format)
        display_df['black_players'] = display_df['black_players'].apply('{:,}'.format)
        display_df['total_wins'] = display_df['total_wins'].apply('{:,}'.format)
        display_df['total_draws'] = display_df['total_draws'].apply('{:,}'.format)
        display_df['total_losses'] = display_df['total_losses'].apply('{:,}'.format)
        
        # Rename columns for better display
        display_df.columns = ['ECO', 'Opening Name', 'Total Games', 'Unique Players',
                             'White Players', 'Black Players', 'Wins', 'Draws', 'Losses',
                             'Win %', 'Draw %', 'Loss %', 'Score %']
        
        print(display_df.to_string(index=False))
        
        # Summary stats
        avg_score = highest_scoring['score_percentage'].mean()
        median_score = highest_scoring['score_percentage'].median()
        min_score = highest_scoring['score_percentage'].min()
        max_score = highest_scoring['score_percentage'].max()
        total_games_in_results = highest_scoring['total_games'].sum()
        
        print(f"\n--- Summary Statistics ---")
        print(f"Average score: {avg_score:.2f}%")
        print(f"Median score: {median_score:.2f}%")
        print(f"Score range: {min_score:.2f}% - {max_score:.2f}%")
        print(f"Total games in these openings: {total_games_in_results:,}")
        print(f"Note: Score = (Wins + 0.5*Draws) / Total Games * 100")

=== TOP 15 HIGHEST SCORING OPENINGS (MIN 100 GAMES) ===


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

ECO                                                      Opening Name Total Games Unique Players White Players Black Players    Wins Draws  Losses  Win %  Draw %  Loss %  Score %
C71                                        Ruy Lopez: Noah's Ark Trap     7,884.0          1,970         1,318           672 4,566.0 393.0 2,925.0  57.91    4.98   37.10    60.41
D00  Blackmar-Diemer Gambit: Bogoljubov Variation, Nimzowitsch Attack       210.0            135            40            95   121.0   8.0    81.0  57.62    3.81   38.57    59.52
C80                                    Ruy Lopez: Open, Karpov Gambit       130.0             41            23            21    75.0   4.0    51.0  57.69    3.08   39.23    59.23
A00                                       Barnes Opening: Fool's Mate       479.0            433           162           271   283.0   0.0   196.0  59.08    0.00   40.92    59.08
A03                   Bird Opening: Dutch Variation, Dudweiler Gambit     4,242.0          3,319         

## 5. Lowest Scoring Openings (Worst Win Rates)

These openings have the lowest win rates, which might indicate they're fundamentally weak or poorly suited for general recommendations.

In [14]:
# Lowest scoring openings (worst win rates) - only include openings with meaningful sample sizes
if db_path.exists():
    with get_db_connection(db_path) as con:
        print(f"=== TOP {TOP_N_LOWEST_SCORING} LOWEST SCORING OPENINGS (MIN 100 GAMES) ===")
        
        lowest_scoring = con.execute(f"""
            SELECT 
                o.eco,
                o.name,
                SUM(pos.num_wins + pos.num_draws + pos.num_losses) as total_games,
                COUNT(DISTINCT pos.player_id) as unique_players,
                COUNT(DISTINCT CASE WHEN pos.color = 'w' THEN pos.player_id END) as white_players,
                COUNT(DISTINCT CASE WHEN pos.color = 'b' THEN pos.player_id END) as black_players,
                SUM(pos.num_wins) as total_wins,
                SUM(pos.num_draws) as total_draws,
                SUM(pos.num_losses) as total_losses,
                ROUND(SUM(pos.num_wins) * 100.0 / NULLIF(SUM(pos.num_wins + pos.num_draws + pos.num_losses), 0), 2) as win_percentage,
                ROUND(SUM(pos.num_draws) * 100.0 / NULLIF(SUM(pos.num_wins + pos.num_draws + pos.num_losses), 0), 2) as draw_percentage,
                ROUND(SUM(pos.num_losses) * 100.0 / NULLIF(SUM(pos.num_wins + pos.num_draws + pos.num_losses), 0), 2) as loss_percentage,
                -- Score calculation: (wins + 0.5*draws) / total_games * 100
                ROUND((SUM(pos.num_wins) + 0.5 * SUM(pos.num_draws)) * 100.0 / NULLIF(SUM(pos.num_wins + pos.num_draws + pos.num_losses), 0), 2) as score_percentage
            FROM opening o
            JOIN player_opening_stats pos ON o.id = pos.opening_id
            GROUP BY o.id, o.eco, o.name
            HAVING SUM(pos.num_wins + pos.num_draws + pos.num_losses) >= 100  -- Only consider openings with meaningful sample size
            ORDER BY score_percentage ASC, total_games DESC
            LIMIT {TOP_N_LOWEST_SCORING}
        """).fetchdf()
        
        # Format the display
        display_df = lowest_scoring.copy()
        display_df['total_games'] = display_df['total_games'].apply('{:,}'.format)
        display_df['unique_players'] = display_df['unique_players'].apply('{:,}'.format)
        display_df['white_players'] = display_df['white_players'].apply('{:,}'.format)
        display_df['black_players'] = display_df['black_players'].apply('{:,}'.format)
        display_df['total_wins'] = display_df['total_wins'].apply('{:,}'.format)
        display_df['total_draws'] = display_df['total_draws'].apply('{:,}'.format)
        display_df['total_losses'] = display_df['total_losses'].apply('{:,}'.format)
        
        # Rename columns for better display
        display_df.columns = ['ECO', 'Opening Name', 'Total Games', 'Unique Players',
                             'White Players', 'Black Players', 'Wins', 'Draws', 'Losses',
                             'Win %', 'Draw %', 'Loss %', 'Score %']
        
        print(display_df.to_string(index=False))
        
        # Summary stats
        avg_score = lowest_scoring['score_percentage'].mean()
        median_score = lowest_scoring['score_percentage'].median()
        min_score = lowest_scoring['score_percentage'].min()
        max_score = lowest_scoring['score_percentage'].max()
        total_games_in_results = lowest_scoring['total_games'].sum()
        
        print(f"\n--- Summary Statistics ---")
        print(f"Average score: {avg_score:.2f}%")
        print(f"Median score: {median_score:.2f}%")
        print(f"Score range: {min_score:.2f}% - {max_score:.2f}%")
        print(f"Total games in these openings: {total_games_in_results:,}")
        print(f"Note: Score = (Wins + 0.5*Draws) / Total Games * 100")

=== TOP 15 LOWEST SCORING OPENINGS (MIN 100 GAMES) ===


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

ECO                                                                     Opening Name Total Games Unique Players White Players Black Players  Wins Draws Losses  Win %  Draw %  Loss %  Score %
C44                                                      Scotch Game: Sea-cadet Mate       155.0            138            45            93  58.0   0.0   97.0  37.42    0.00   62.58    37.42
C39 King's Gambit Accepted: Kieseritzky Gambit, Berlin Defense, de Riviere Variation       270.0            172           114            58  99.0   7.0  164.0  36.67    2.59   60.74    37.96
C96                             Ruy Lopez: Closed, Chigorin Defense, Gajewski Gambit       136.0             73            63            10  50.0   8.0   78.0  36.76    5.88   57.35    39.71
C23                                                   Bishop's Opening: Lopez Gambit       143.0            125            35            90  56.0   2.0   85.0  39.16    1.40   59.44    39.86
B63  Sicilian Defense: Richter-Rauzer Variati

## 6. Distribution Analysis

Let's look at the overall distribution of games and players to understand what thresholds might make sense for filtering.

In [15]:
# Distribution analysis - understand the spread of data to inform filtering decisions
if db_path.exists():
    with get_db_connection(db_path) as con:
        print("=== OPENING DISTRIBUTION ANALYSIS ===")
        
        # Get distribution statistics for all openings
        distribution_stats = con.execute("""
            SELECT 
                COUNT(*) as total_openings,
                MIN(total_games) as min_games,
                MAX(total_games) as max_games,
                ROUND(AVG(total_games), 1) as avg_games,
                ROUND(MEDIAN(total_games), 1) as median_games,
                ROUND(STDDEV(total_games), 1) as stddev_games,
                MIN(unique_players) as min_players,
                MAX(unique_players) as max_players,
                ROUND(AVG(unique_players), 1) as avg_players,
                ROUND(MEDIAN(unique_players), 1) as median_players,
                ROUND(STDDEV(unique_players), 1) as stddev_players
            FROM (
                SELECT 
                    o.id,
                    SUM(pos.num_wins + pos.num_draws + pos.num_losses) as total_games,
                    COUNT(DISTINCT pos.player_id) as unique_players
                FROM opening o
                JOIN player_opening_stats pos ON o.id = pos.opening_id
                GROUP BY o.id
            ) stats
        """).fetchone()
        
        print("--- All Openings Statistics ---")
        print(f"Total Openings: {distribution_stats[0]:,}")
        print(f"Games per Opening - Min: {distribution_stats[1]:,}, Max: {distribution_stats[2]:,}, Avg: {distribution_stats[3]:,}, Median: {distribution_stats[4]:,}")
        print(f"Players per Opening - Min: {distribution_stats[5]:,}, Max: {distribution_stats[6]:,}, Avg: {distribution_stats[7]:,}, Median: {distribution_stats[8]:,}")
        
        # Percentile analysis for games
        print("\n--- Games per Opening Percentiles ---")
        games_percentiles = con.execute("""
            SELECT 
                ROUND(PERCENTILE_CONT(0.1) WITHIN GROUP (ORDER BY total_games), 1) as p10,
                ROUND(PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY total_games), 1) as p25,
                ROUND(PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY total_games), 1) as p50,
                ROUND(PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY total_games), 1) as p75,
                ROUND(PERCENTILE_CONT(0.9) WITHIN GROUP (ORDER BY total_games), 1) as p90,
                ROUND(PERCENTILE_CONT(0.95) WITHIN GROUP (ORDER BY total_games), 1) as p95,
                ROUND(PERCENTILE_CONT(0.99) WITHIN GROUP (ORDER BY total_games), 1) as p99
            FROM (
                SELECT SUM(pos.num_wins + pos.num_draws + pos.num_losses) as total_games
                FROM opening o
                JOIN player_opening_stats pos ON o.id = pos.opening_id
                GROUP BY o.id
            ) stats
        """).fetchone()
        
        percentile_labels = ['10th', '25th', '50th (Median)', '75th', '90th', '95th', '99th']
        for i, label in enumerate(percentile_labels):
            print(f"{label}: {games_percentiles[i]:,} games")
        
        # Percentile analysis for players
        print("\n--- Players per Opening Percentiles ---")
        players_percentiles = con.execute("""
            SELECT 
                ROUND(PERCENTILE_CONT(0.1) WITHIN GROUP (ORDER BY unique_players), 1) as p10,
                ROUND(PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY unique_players), 1) as p25,
                ROUND(PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY unique_players), 1) as p50,
                ROUND(PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY unique_players), 1) as p75,
                ROUND(PERCENTILE_CONT(0.9) WITHIN GROUP (ORDER BY unique_players), 1) as p90,
                ROUND(PERCENTILE_CONT(0.95) WITHIN GROUP (ORDER BY unique_players), 1) as p95,
                ROUND(PERCENTILE_CONT(0.99) WITHIN GROUP (ORDER BY unique_players), 1) as p99
            FROM (
                SELECT COUNT(DISTINCT pos.player_id) as unique_players
                FROM opening o
                JOIN player_opening_stats pos ON o.id = pos.opening_id
                GROUP BY o.id
            ) stats
        """).fetchone()
        
        for i, label in enumerate(percentile_labels):
            print(f"{label}: {players_percentiles[i]:,} players")
        
        # Count openings by game thresholds
        print("\n--- Openings by Game Count Thresholds ---")
        game_thresholds = [1, 10, 50, 100, 500, 1000, 5000, 10000]
        for threshold in game_thresholds:
            count = con.execute(f"""
                SELECT COUNT(*)
                FROM (
                    SELECT SUM(pos.num_wins + pos.num_draws + pos.num_losses) as total_games
                    FROM opening o
                    JOIN player_opening_stats pos ON o.id = pos.opening_id
                    GROUP BY o.id
                    HAVING total_games < {threshold}
                ) stats
            """).fetchone()[0]
            percentage = (count / distribution_stats[0]) * 100
            print(f"Openings with < {threshold:,} games: {count:,} ({percentage:.1f}%)")
        
        # Count openings by player thresholds
        print("\n--- Openings by Player Count Thresholds ---")
        player_thresholds = [1, 5, 10, 25, 50, 100, 500, 1000]
        for threshold in player_thresholds:
            count = con.execute(f"""
                SELECT COUNT(*)
                FROM (
                    SELECT COUNT(DISTINCT pos.player_id) as unique_players
                    FROM opening o
                    JOIN player_opening_stats pos ON o.id = pos.opening_id
                    GROUP BY o.id
                    HAVING unique_players < {threshold}
                ) stats
            """).fetchone()[0]
            percentage = (count / distribution_stats[0]) * 100
            print(f"Openings with < {threshold:,} players: {count:,} ({percentage:.1f}%)")

=== OPENING DISTRIBUTION ANALYSIS ===
--- All Openings Statistics ---
Total Openings: 3,593
Games per Opening - Min: 1, Max: 11,806,290, Avg: 158,334.2, Median: 10,064.0
Players per Opening - Min: 640,532.8, Max: 1, Avg: 49,996, Median: 6,749.1

--- Games per Opening Percentiles ---
10th: 157.0 games
25th: 1,270.0 games
50th (Median): 10,064.0 games
75th: 62,039.0 games
90th: 302,352.6 games
95th: 686,576.4 games
99th: 2,882,848.7 games

--- Players per Opening Percentiles ---
--- All Openings Statistics ---
Total Openings: 3,593
Games per Opening - Min: 1, Max: 11,806,290, Avg: 158,334.2, Median: 10,064.0
Players per Opening - Min: 640,532.8, Max: 1, Avg: 49,996, Median: 6,749.1

--- Games per Opening Percentiles ---
10th: 157.0 games
25th: 1,270.0 games
50th (Median): 10,064.0 games
75th: 62,039.0 games
90th: 302,352.6 games
95th: 686,576.4 games
99th: 2,882,848.7 games

--- Players per Opening Percentiles ---
10th: 95.2 players
25th: 497.0 players
50th (Median): 2,636.0 players
75th

## 7. Recommendations for Filtering

Based on the analysis above, here are some recommendations for filtering unneeded openings from the database.

In [16]:
# Generate filtering recommendations based on the analysis
if db_path.exists():
    with get_db_connection(db_path) as con:
        print("=== FILTERING RECOMMENDATIONS ===")
        
        # Calculate some threshold recommendations
        total_openings = con.execute('SELECT COUNT(*) FROM opening').fetchone()[0]
        
        # Recommendation 1: Filter by minimum games
        min_games_threshold = 50  # Adjust based on your needs
        openings_below_games_threshold = con.execute(f"""
            SELECT COUNT(*)
            FROM (
                SELECT SUM(pos.num_wins + pos.num_draws + pos.num_losses) as total_games
                FROM opening o
                JOIN player_opening_stats pos ON o.id = pos.opening_id
                GROUP BY o.id
                HAVING total_games < {min_games_threshold}
            ) stats
        """).fetchone()[0]
        
        games_filter_percentage = (openings_below_games_threshold / total_openings) * 100
        
        print(f"1. MINIMUM GAMES FILTER (< {min_games_threshold} games):")
        print(f"   - Would remove {openings_below_games_threshold:,} openings ({games_filter_percentage:.1f}% of total)")
        print(f"   - Rationale: Too few games to provide reliable recommendations")
        
        # Recommendation 2: Filter by minimum players
        min_players_threshold = 10  # Adjust based on your needs
        openings_below_players_threshold = con.execute(f"""
            SELECT COUNT(*)
            FROM (
                SELECT COUNT(DISTINCT pos.player_id) as unique_players
                FROM opening o
                JOIN player_opening_stats pos ON o.id = pos.opening_id
                GROUP BY o.id
                HAVING unique_players < {min_players_threshold}
            ) stats
        """).fetchone()[0]
        
        players_filter_percentage = (openings_below_players_threshold / total_openings) * 100
        
        print(f"\n2. MINIMUM PLAYERS FILTER (< {min_players_threshold} unique players):")
        print(f"   - Would remove {openings_below_players_threshold:,} openings ({players_filter_percentage:.1f}% of total)")
        print(f"   - Rationale: Too few players to generalize recommendations")
        
        # Recommendation 3: Filter by extreme win rates
        extreme_win_rate_threshold = 80.0  # Adjust based on your needs
        extreme_high_win_rate = con.execute(f"""
            SELECT COUNT(*)
            FROM (
                SELECT 
                    ROUND(SUM(pos.num_wins) * 100.0 / NULLIF(SUM(pos.num_wins + pos.num_draws + pos.num_losses), 0), 2) as win_percentage
                FROM opening o
                JOIN player_opening_stats pos ON o.id = pos.opening_id
                GROUP BY o.id
                HAVING SUM(pos.num_wins + pos.num_draws + pos.num_losses) >= 100
                AND win_percentage > {extreme_win_rate_threshold}
            ) stats
        """).fetchone()[0]
        
        extreme_low_win_rate = con.execute(f"""
            SELECT COUNT(*)
            FROM (
                SELECT 
                    ROUND(SUM(pos.num_wins) * 100.0 / NULLIF(SUM(pos.num_wins + pos.num_draws + pos.num_losses), 0), 2) as win_percentage
                FROM opening o
                JOIN player_opening_stats pos ON o.id = pos.opening_id
                GROUP BY o.id
                HAVING SUM(pos.num_wins + pos.num_draws + pos.num_losses) >= 100
                AND win_percentage < {100 - extreme_win_rate_threshold}
            ) stats
        """).fetchone()[0]
        
        extreme_total = extreme_high_win_rate + extreme_low_win_rate
        extreme_filter_percentage = (extreme_total / total_openings) * 100
        
        print(f"\n3. EXTREME WIN RATE FILTER (>{extreme_win_rate_threshold}% or <{100-extreme_win_rate_threshold}% win rate, min 100 games):")
        print(f"   - Would remove {extreme_total:,} openings ({extreme_filter_percentage:.1f}% of total)")
        print(f"     * High win rate (>{extreme_win_rate_threshold}%): {extreme_high_win_rate:,}")
        print(f"     * Low win rate (<{100-extreme_win_rate_threshold}%): {extreme_low_win_rate:,}")
        print(f"   - Rationale: Likely unbalanced or situational openings")
        
        # Combined filter impact
        combined_filter = con.execute(f"""
            SELECT COUNT(DISTINCT o.id)
            FROM opening o
            JOIN player_opening_stats pos ON o.id = pos.opening_id
            GROUP BY o.id
            HAVING 
                SUM(pos.num_wins + pos.num_draws + pos.num_losses) < {min_games_threshold}
                OR COUNT(DISTINCT pos.player_id) < {min_players_threshold}
                OR (
                    SUM(pos.num_wins + pos.num_draws + pos.num_losses) >= 100
                    AND (
                        SUM(pos.num_wins) * 100.0 / NULLIF(SUM(pos.num_wins + pos.num_draws + pos.num_losses), 0) > {extreme_win_rate_threshold}
                        OR SUM(pos.num_wins) * 100.0 / NULLIF(SUM(pos.num_wins + pos.num_draws + pos.num_losses), 0) < {100 - extreme_win_rate_threshold}
                    )
                )
        """).fetchone()[0]
        
        combined_percentage = (combined_filter / total_openings) * 100
        
        print(f"\n4. COMBINED FILTER IMPACT:")
        print(f"   - Would remove {combined_filter:,} openings ({combined_percentage:.1f}% of total)")
        print(f"   - Remaining openings: {total_openings - combined_filter:,} ({100 - combined_percentage:.1f}% of total)")
        
        print(f"\n=== SUGGESTED THRESHOLDS FOR YOUR USE CASE ===")
        print(f"Minimum Games: {min_games_threshold} (adjust based on desired data quality)")
        print(f"Minimum Players: {min_players_threshold} (adjust based on generalization needs)")
        print(f"Win Rate Bounds: {100-extreme_win_rate_threshold}% - {extreme_win_rate_threshold}% (adjust based on balance requirements)")
        print(f"\nNote: You can modify the threshold variables at the top of this cell to experiment with different values.")

=== FILTERING RECOMMENDATIONS ===
1. MINIMUM GAMES FILTER (< 50 games):
   - Would remove 203 openings (5.6% of total)
   - Rationale: Too few games to provide reliable recommendations

2. MINIMUM PLAYERS FILTER (< 10 unique players):
   - Would remove 82 openings (2.3% of total)
   - Rationale: Too few players to generalize recommendations

3. EXTREME WIN RATE FILTER (>80.0% or <20.0% win rate, min 100 games):
   - Would remove 0 openings (0.0% of total)
     * High win rate (>80.0%): 0
     * Low win rate (<20.0%): 0
   - Rationale: Likely unbalanced or situational openings

2. MINIMUM PLAYERS FILTER (< 10 unique players):
   - Would remove 82 openings (2.3% of total)
   - Rationale: Too few players to generalize recommendations

3. EXTREME WIN RATE FILTER (>80.0% or <20.0% win rate, min 100 games):
   - Would remove 0 openings (0.0% of total)
     * High win rate (>80.0%): 0
     * Low win rate (<20.0%): 0
   - Rationale: Likely unbalanced or situational openings

4. COMBINED FILTER